# 이미지 활용한 CNN 모델 만들기
- ImageFolder
- transforms
- DataLoader
- CNN
- Weight

사용하기

# 이미지 = 해파리.

In [43]:
# 모듈 로딩

from PIL import Image
import torchvision.transforms as transforms
import cv2
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn.functional as f
import torchmetrics.functional as metrics

In [31]:
# 사이즈 조정, 텐서화
preprocessing = transforms.Compose([transforms.Resize(size=(50,50)), transforms.ToTensor()])

In [32]:
# 폴명이 분류 클래스 되는 데이터셋 만들기
imgroot = '../../../datas/Project Jellyfish/train_Test_Valid/Train'
imgdataset = ImageFolder(root=imgroot, transform=preprocessing)
dataloader = DataLoader(dataset=imgdataset, batch_size=10, shuffle=True)

In [53]:
# valid dataset
valroot = '../../../datas/Project Jellyfish/train_Test_Valid/valid'
valdataset = ImageFolder(root=imgroot, transform=preprocessing)
valid_dl = DataLoader(dataset=imgdataset, batch_size=10, shuffle=True)

In [54]:
# test dataset

testroot = '../../../datas/Project Jellyfish/train_Test_Valid/test'
testdataset = ImageFolder(root=imgroot, transform=preprocessing)
test_dl = DataLoader(dataset=imgdataset, batch_size=10, shuffle=True)

In [33]:
# 확인
imgdataset.classes, imgdataset.class_to_idx, imgdataset.imgs

(['Moon_jellyfish',
  'barrel_jellyfish',
  'blue_jellyfish',
  'compass_jellyfish',
  'lions_mane_jellyfish',
  'mauve_stinger_jellyfish'],
 {'Moon_jellyfish': 0,
  'barrel_jellyfish': 1,
  'blue_jellyfish': 2,
  'compass_jellyfish': 3,
  'lions_mane_jellyfish': 4,
  'mauve_stinger_jellyfish': 5},
 [('../../../datas/Project Jellyfish/train_Test_Valid/Train\\Moon_jellyfish\\01.jpg',
   0),
  ('../../../datas/Project Jellyfish/train_Test_Valid/Train\\Moon_jellyfish\\02.jpg',
   0),
  ('../../../datas/Project Jellyfish/train_Test_Valid/Train\\Moon_jellyfish\\03.jpg',
   0),
  ('../../../datas/Project Jellyfish/train_Test_Valid/Train\\Moon_jellyfish\\04.jpg',
   0),
  ('../../../datas/Project Jellyfish/train_Test_Valid/Train\\Moon_jellyfish\\05.jpg',
   0),
  ('../../../datas/Project Jellyfish/train_Test_Valid/Train\\Moon_jellyfish\\06.jpg',
   0),
  ('../../../datas/Project Jellyfish/train_Test_Valid/Train\\Moon_jellyfish\\08.jpg',
   0),
  ('../../../datas/Project Jellyfish/train_Test_V

In [34]:
# 전처리
for(img, label) in dataloader:
    print(img.shape, label)

torch.Size([10, 3, 50, 50]) tensor([2, 2, 5, 2, 2, 5, 0, 1, 0, 5])
torch.Size([10, 3, 50, 50]) tensor([2, 2, 0, 4, 0, 1, 0, 3, 3, 2])
torch.Size([10, 3, 50, 50]) tensor([4, 1, 5, 5, 2, 4, 1, 0, 3, 0])
torch.Size([10, 3, 50, 50]) tensor([3, 5, 3, 5, 5, 2, 1, 3, 0, 2])
torch.Size([10, 3, 50, 50]) tensor([5, 4, 3, 1, 4, 4, 1, 5, 4, 4])
torch.Size([10, 3, 50, 50]) tensor([5, 4, 3, 0, 3, 5, 2, 4, 3, 1])
torch.Size([10, 3, 50, 50]) tensor([2, 4, 0, 5, 1, 1, 3, 0, 3, 0])
torch.Size([10, 3, 50, 50]) tensor([4, 5, 0, 0, 3, 5, 1, 1, 1, 4])
torch.Size([10, 3, 50, 50]) tensor([3, 0, 2, 5, 2, 1, 4, 0, 2, 5])
torch.Size([10, 3, 50, 50]) tensor([5, 2, 5, 4, 4, 1, 0, 2, 2, 3])
torch.Size([10, 3, 50, 50]) tensor([1, 4, 2, 3, 5, 5, 1, 4, 4, 3])
torch.Size([10, 3, 50, 50]) tensor([3, 0, 0, 1, 4, 0, 4, 2, 1, 1])
torch.Size([10, 3, 50, 50]) tensor([0, 2, 3, 3, 2, 5, 5, 0, 2, 0])
torch.Size([10, 3, 50, 50]) tensor([4, 4, 4, 4, 3, 3, 5, 2, 4, 5])
torch.Size([10, 3, 50, 50]) tensor([1, 1, 3, 0, 0, 4, 2, 2, 5,

In [35]:
for (img, label) in dataloader:
    print(img.shape, label.shape, label.bincount())

torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([1, 1, 1, 2, 1, 4])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([1, 2, 2, 2, 1, 2])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([2, 1, 2, 3, 1, 1])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([1, 1, 3, 1, 3, 1])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([2, 0, 3, 1, 1, 3])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([3, 2, 1, 1, 1, 2])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([2, 0, 1, 2, 3, 2])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([5, 1, 3, 0, 1])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([3, 1, 0, 1, 2, 3])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([1, 2, 2, 2, 0, 3])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([2, 0, 2, 1, 2, 3])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([2, 2, 2, 2, 1, 1])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([0, 2, 2, 2, 2, 2])
torch.Size([10, 3, 50, 50]) torch.Size([10]) tensor([2, 0, 2, 3, 2,

In [36]:
# 첫번째 conv 레이어
conlayer = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
for img, label in dataloader:
    output = conlayer(img)
    print(output.shape)
    break

torch.Size([10, 16, 50, 50])


In [37]:
# Pooling layer

pool = nn.MaxPool2d(kernel_size=2)
relu = nn.ReLU()(output)
pool_output = pool(relu)
print(pool_output.shape)

torch.Size([10, 16, 25, 25])


In [39]:
# 완전 연결 층 추가
# 특징 맵 평탄화하기
flatten = output.view(output.size(0),-1)
fc_layer = nn.Linear(flatten.size(1), len(label))

In [40]:
# 손실 함수 정의
c = nn.CrossEntropyLoss()
import torch.optim as optim
# optimizer:
optimizer = optim.Adam(conlayer.parameters(), lr=0.001)

In [44]:
EPOCHS = 42
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)
batch_size = 10

In [41]:
class JELLYFISHMODEL(nn.Module):   
    def __init__(self, in_, out_):
        super().__init__()
        self.layer1 = nn.Linear(in_,120)
        self.layer2 = nn.Linear(120,84)
        self.layer3 = nn.Linear(84,out_)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        y = self.layer1(x)
        y = self.relu(y)
        y = self.layer2(y)
        y = self.relu(y)
        y = self.layer3(y)
        return y

In [48]:
# Model instance
model = JELLYFISHMODEL(in_=3, out_=10)

In [60]:
# 학습
def training(epoch):
    # 학습모드
    model.train()
    
    #배치크기만큼 학습 진행하고 저장하기
    trainreport = [[],[],[]]
    for idx,(feature, target)in enumerate(dataloader):
        #배치크기만큼 학습 데이터 준비
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        feature = feature.unsqueeze(0)
        #학습
        pretarget = model(feature)
        
        # 손실
        loss = LOSS_FN(pretarget, target)
        trainreport[0].append(loss.item())
        
        # 성능평가
        acc = metrics.accuracy(pretarget, target, task='multiclass',num_classes = len(label))
        trainreport[1].append(acc)
        
        f1 = metrics.f1_score(pretarget, target, task='multiclass',num_classes = len(label))
        trainreport[2].append(f1)
        
        #ep 단위 학습 진행 메시지 출력
        loss = (sum(trainreport[0]/batch_size)).item()
        acc = (sum(trainreport[1]/batch_size)).item()
        f1 = (sum(trainreport[2]/batch_size)).item()
        
        print(f'[{epoch}] loss: {loss:.3f}, acc: {acc:.3f}, f1: {f1:.3f}')
        
        return loss,acc,f1

In [61]:
# 검증 및 테스트 진행함수
def testing(epoch, kind='val'):
    # 학습
    model.eval()
    
    with torch.no_grad():
        test_report= [[],[],[]]
        if kind=='val':
            dataloader = valid_dl 

        else:
            test_dl
        for idx, (feature, target) in enumerate(test_dl):
            #배치크기만큼 학습데이터 준비
            feature, target= feature.to(DEVICE) , target.to(DEVICE)
            
            # 학습
            pre_target = model(feature)
            

            # 손실계산
            loss = LOSS_FN(pre_target, target)
            test_report[0].append(loss)
            
            # 성능 평가 
            acc = metrics.accuracy(pre_target, target, task='multiclass', num_classes=len(label))
            test_report[1].append(acc)
            
            f1 = metrics.f1_score(pre_target, target, task='multiclass', num_classes=len(label))
            test_report[2].append(f1)
            
             
        #ep 단위 학습 진행 메시지 출력
        loss = (sum(test_report[0]/batch_size)).item()
        acc = (sum(test_report[1]/batch_size)).item()
        f1 = (sum(test_report[2]/batch_size)).item()
        
        print(f'[{epoch}] loss: {loss:.3f}, acc: {acc:.3f}, f1: {f1:.3f}')
        
        return loss,acc,f1

In [62]:
# 실행 디바이스
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
#입출력 피처
IN_, OUT_ = 500, 10
#최적화 인스턴스 생성
OPTIMIZER = optim.Adam(model.parameters(), lr=0.001)
#손실함수
LOSS_FN = nn.CrossEntropyLoss()
#학습 횟수 설정
EPOCHS = 10

In [63]:
# 학습 데이별 성능지표 값
train_ = {'loss':[], 'acc':[], 'f1':[]}
val_ = {'loss':[], 'acc':[], 'f1':[]}

#모델 저장 관련 변수
pre_va_score=0

#학습 진행
for epo in range(EPOCHS):
    print(f"[Epoch {epo+1}/{EPOCHS}] ", end='')
    tr_score=training(epo)
    va_score=testing(epo)
    
    # 검증 데이터 기준 학습된 모델 저장하기: Loss가 이전보다 작아진 경우 저장/ acc 또는 F1이면 이전보다 큰 경우 저장
    # if pre_va_score > va_score[0]:
    #     torch.save(model, filename)
    # 가중치저장하기
    #     model.state_dict()
    
    for idx, key in enumerate(train_):
        train_[key].append(tr_score[idx])
        val_[key].append(va_score[idx])

print("Done!")

[Epoch 1/10] 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1500x50 and 3x120)